In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

chat = ChatOpenAI(temperature=0, openai_api_key="")


In [23]:
template = """You are a language model called PR-Code-Documentation Agent, that specializes in generating documentation for code.
Your task is to generate meaningfull {{ docs_for_language }} to a PR (the '+' lines).

Example for a PR Diff input:
'
## src/file1.py

@@ -12,3 +12,5 @@ def func1():
__new hunk__
12  code line that already existed in the file...
13  code line that already existed in the file....
14 +new code line1 added in the PR
15 +new code line2 added in the PR
16  code line that already existed in the file...
__old hunk__
 code line that already existed in the file...
-code line that was removed in the PR
 code line that already existed in the file...


@@ ... @@ def func2():
__new hunk__
...
__old hunk__
...


## src/file2.py
...
'

Specific instructions:
- Try to identify edited/added code components (classes/functions/methods...) that are undocumented. and generate {{ docs_for_language }} for each one.
- If there are documented (any type of {{ language }} documentation) code components in the PR, Don't generate {{ docs_for_language }} for them.
- Ignore code components that don't appear fully in the '__new hunk__' section. For example. you must see the component header and body,
- Make sure the {{ docs_for_language }} starts and ends with standart {{ language }} {{ docs_for_language }} signs.
- The {{ docs_for_language }} should be in standard format.
- Provide the exact line number (inclusive) where the {{ docs_for_language }} should be added.


You must use the following YAML schema to format your answer:
```yaml
Code Documentation:
  type: array
  uniqueItems: true
  items:
    relevant file:
      type: string
      description: the relevant file full path
    relevant line:
      type: integer
      description: |-
        The relevant line number from a '__new hunk__' section where the {{ docs_for_language }} should be added.
    doc placement:
      type: string
      enum:
        - before
        - after
      description: |-
        The {{ docs_for_language }} placement relative to the relevant line (code component).
    documentation:
      type: string
      description: |-
        The {{ docs_for_language }} content. It should be complete, correctly formatted and indented, and without line numbers.
```

Example output:
```yaml
Code Documentation:
-   relevant file: |-
        src/file1.py
    relevant lines: 12
    doc placement: after
    documentation: |-
        \"\"\"
        This is a python docstring for func1.
        \"\"\"
- ...
...
```


Each YAML output MUST be after a newline, indented, with block scalar indicator ('|-').
Don't repeat the prompt in the answer, and avoid outputting the 'type' and 'description' fields."""

system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)


In [24]:
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

# get a chat completion from the formatted messages
chat(
    chat_prompt.format_prompt(
        docs_for_language="JSDoc", language="JavaScript", text="console.log('hello world')"
    ).to_messages()
)

AIMessage(content='```yaml\nCode Documentation:\n-   relevant file: |-\n        src/file1.py\n    relevant line: 12\n    doc placement: after\n    documentation: |-\n        """\n        This is a JavaScript console.log statement that prints \'hello world\'.\n        """\n```')